In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from typing import List
import time
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def extract_urls_lists(search_page_url: str) -> List[str]:
    response = requests.get(search_page_url, params = {})
    soup = BeautifulSoup(response.text, 'html.parser')

    list_of_urls=[]

    for article in soup.find_all('iw-search-card-rendered'):
        for link in article.find_all('a'):
            list_of_urls.append(link.get('href'))
    return list_of_urls

def extract_x_urls(number_of_urls: int=30) -> List[str]:
    immo_base_url = "https://www.immoweb.be/en/search/house/for-sale?countries=BE&page="
    immo_end_url = "&orderBy=relevance"
    searchpages_urls_list =[]

    for index in range(1,round(min(334,number_of_urls/30 + 1))):
        full_url = immo_base_url + str(index) + immo_end_url
        searchpages_urls_list.append(full_url)
    
    full_urls_list =[]

    with ThreadPoolExecutor() as pool:
        full_urls_list=list(pool.map(extract_urls_lists, searchpages_urls_list))  

    if number_of_urls>9990:
        immo_base_url = "https://www.immoweb.be/en/search/apartment/for-sale?countries=BE&page="
        immo_end_url = "&orderBy=relevance"
        searchpages_urls_list =[]

        for index in range(1,round(min(334,(number_of_urls-9990)/30 + 1))):
            full_url = immo_base_url + str(index) + immo_end_url
            searchpages_urls_list.append(full_url)

        with ThreadPoolExecutor() as pool:
            full_urls_list+=list(pool.map(extract_urls_lists, searchpages_urls_list))  
            
    return list(set(sum(full_urls_list,[])))


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

def make_one_data_frame(url):
  try:
    response = requests.get(url)
    test_page = response.text

    dfs = pd.read_html(test_page)

    full_df = pd.concat(dfs).dropna(thresh=2).T
    full_df.columns = full_df.iloc[0]
    full_df = full_df[1:]
    full_df = full_df.loc[:, full_df.columns.isin(['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace'])]

    return full_df
  except:
    return ['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace']

In [ ]:
list_of_urls = extract_x_urls(15000)
print('list ready')

In [ ]:
dataframes_list = []
with ThreadPoolExecutor() as pool:
    results = list(pool.map(make_one_data_frame, list_of_urls[1:10000]))
    for result in results:
        dataframes_list.append(result)

index = 0
for element in dataframes_list:
    if type(element) != pandas.core.frame.DataFrame:
        dataframes_list.pop(index)
    index += 1
    
full_df = pd.concat(dataframes_list).reset_index(drop=True)
full_df.dropna(thresh=2,inplace=True)

print('done extracting')

full_df.to_csv("df.csv")
#full_df